# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port hedland,-20.3167,118.5667,30.518,20,12,9.77,AU,1713927646
1,1,papatowai,-46.5619,169.4708,28.982,65,100,1.59,NZ,1713927648
2,2,iqaluit,63.7506,-68.5145,27.000,79,100,2.06,CA,1713927648
3,3,bredasdorp,-34.5322,20.0403,28.838,96,100,0.38,ZA,1713927651
4,4,zalingei,12.9096,23.4741,29.921,14,6,2.03,SD,1713927651


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

map_plot = city_data_df.hvplot.points('Lng', 'Lat', 
                                      size='Humidity', 
                                      title='City Map with Humidity')

# Show the plot
map_plot.opts(width = 600, height = 400)

:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values

# Define your ideal weather conditions
ideal_conditions = {
    'max_temp': (22, 28),  # Max temperature between 22 and 30 degrees
    'wind_speed': 10,   # Wind speed less than 10 m/s
    'cloudiness': 0    # Zero cloudiness
}

# Narrow down the DataFrame based on ideal conditions and drop rows with null values
filtered_df = city_data_df.dropna().loc[
    (city_data_df['Max Temp'] > ideal_conditions['max_temp'][0]) & 
    (city_data_df['Max Temp'] < ideal_conditions['max_temp'][1]) & 
    (city_data_df['Wind Speed'] < ideal_conditions['wind_speed']) & 
    (city_data_df['Cloudiness'] == ideal_conditions['cloudiness'])
]

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,84,ushuaia,-54.8000,-68.3000,27.896,75,0,1.03,AR,1713927606
119,119,thunder bay,48.4001,-89.3168,27.314,55,0,2.57,CA,1713927719
131,131,bemidji,47.4736,-94.8803,27.921,49,0,0.00,US,1713927722
145,145,longyearbyen,78.2186,15.6401,26.706,79,0,5.14,SJ,1713927551
284,284,yarmouth,41.7057,-70.2286,27.927,73,0,0.89,US,1713927551
315,315,egilsstadir,65.2653,-14.3948,27.110,93,0,2.06,IS,1713927748
490,490,duffel,51.0958,4.5118,27.878,87,0,4.63,BE,1713927770
530,530,khanty-mansiysk,61.0042,69.0019,26.121,67,0,3.00,RU,1713927774
564,564,zemetchino,53.4973,42.6163,27.947,82,0,2.49,RU,1713927778
569,569,akureyri,65.6835,-18.0878,27.211,93,0,1.03,IS,1713927778


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
84,ushuaia,AR,-54.8000,-68.3000,75,
119,thunder bay,CA,48.4001,-89.3168,55,
131,bemidji,US,47.4736,-94.8803,49,
145,longyearbyen,SJ,78.2186,15.6401,79,
284,yarmouth,US,41.7057,-70.2286,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "radius": radius,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
bemidji - nearest hotel: Hampton Inn & Suites
longyearbyen - nearest hotel: The Vault
yarmouth - nearest hotel: The Inn At Cape Cod
egilsstadir - nearest hotel: Valaskjálf
duffel - nearest hotel: Pure
khanty-mansiysk - nearest hotel: Гамма
zemetchino - nearest hotel: Отель
akureyri - nearest hotel: Hótel Norðurland


,City,Country,Lat,Lng,Humidity,Hotel Name
84,ushuaia,AR,-54.8000,-68.3000,75,Apart Hotel Aires del Beagle
119,thunder bay,CA,48.4001,-89.3168,55,Hampton Inn & Suites by Hilton Thunder Bay
131,bemidji,US,47.4736,-94.8803,49,Hampton Inn & Suites
145,longyearbyen,SJ,78.2186,15.6401,79,The Vault
284,yarmouth,US,41.7057,-70.2286,73,The Inn At Cape Cod
315,egilsstadir,IS,65.2653,-14.3948,93,Valaskjálf
490,duffel,BE,51.0958,4.5118,87,Pure
530,khanty-mansiysk,RU,61.0042,69.0019,67,Гамма
564,zemetchino,RU,53.4973,42.6163,82,Отель
569,akureyri,IS,65.6835,-18.0878,93,Hótel Norðurland


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)